# Overview

Hay dos tipos cd Cross-Entropy, Binaria y Multiple, BCE o simplemente CE.

Se usa cuando predices una probabilidad.

Se puede calcular con logits o tras la FA, pero computacionalmente, es mas eficiente hacerlo con logits.


Un logit es el resutlado de la red en la capa de clasificacion final antes la  de la funcion de activacion.

¿Que hace?

No mide “distancia”, sino verosimilitud.

Penaliza más los errores muy confiados.

Hace que el modelo aprenda a producir buenas probabilidades, no solo valores cercanos a 0 o 1.

## BCE (Binary Cross-Entropy)

Se usa cuando predices una probabilidad de clase (0 o 1)

Clasificación binaria: salida de una Sigmoid.

Es perro (1) o no es perro (0).

Spam o no spam.

Falla el sistema o no falla.





In [3]:
import torch
import torch.nn.functional as F

# === 1. Logits de ejemplo (salida cruda de la última capa, antes de la sigmoide) ===
logits = torch.tensor([[0.2], [1.5], [-1.2]])  # (3,1) → valores antes de Sigmoid

# === 2. Etiquetas reales ===
y_true = torch.tensor([[0.0], [1.0], [0.0]])   # (3,1)

# === 3. Probabilidades después de aplicar la Sigmoid ===
probs = torch.sigmoid(logits)

print("Logits (z):\n", logits)
print("Probabilidades (σ(z)):\n", probs)
print("Shapes → logits:", tuple(logits.shape), "| y_true:", tuple(y_true.shape))

# === 4. Cross-Entropy calculada a mano (usando probabilidades) ===
# Fórmula: -[ y*log(p) + (1-y)*log(1-p) ]
bce_manual = - (y_true * torch.log(probs) + (1 - y_true) * torch.log(1 - probs))
bce_manual_mean = bce_manual.mean()

print("\nBinary Cross-Entropy (manual):", bce_manual_mean.item())

# === 5. Cross-Entropy con PyTorch ===
# a) usando las probabilidades directamente
bce_probs = F.binary_cross_entropy(probs, y_true)
# b) usando los logits directamente (más estable)
bce_logits = F.binary_cross_entropy_with_logits(logits, y_true)

print("BCE con probabilidades :", bce_probs.item())
print("BCE con logits (estable):", bce_logits.item())

# === 6. Comprobación de equivalencia numérica aproximada ===
assert torch.allclose(bce_probs, bce_logits, atol=1e-6)


Logits (z):
 tensor([[ 0.2000],
        [ 1.5000],
        [-1.2000]])
Probabilidades (σ(z)):
 tensor([[0.5498],
        [0.8176],
        [0.2315]])
Shapes → logits: (3, 1) | y_true: (3, 1)

Binary Cross-Entropy (manual): 0.42094483971595764
BCE con probabilidades : 0.42094483971595764
BCE con logits (estable): 0.42094483971595764


## CE Multiple

Es similar, pero para tareas de clasificacion con mas de 2 clases.


In [ ]:
import torch
import torch.nn.functional as F

# === 1. Logits de ejemplo (salida cruda antes del Softmax) ===
# Supongamos 3 muestras (ya procesadas: logits antes de softmax) y 3 clases (cada fila = muestra)
logits = torch.tensor([
    [2.0, 1.0, 0.1],   # muestra 1
    [0.5, 2.5, 0.3],   # muestra 2
    [0.1, 0.2, 3.0]    # muestra 3
])

# === 2. Etiquetas reales ===
# En multiclase, los targets son índices enteros (0, 1, 2)
y_true = torch.tensor([0, 1, 2])

print("Shapes → logits:", tuple(logits.shape), "| y_true:", tuple(y_true.shape))

# === 3. Probabilidades después del Softmax (solo para inspección) ===
probs = torch.softmax(logits, dim=1)
print("\nProbabilidades (Softmax):\n", probs)

# === 4. Cross-Entropy con PyTorch (logits → CE estable) ===
ce_loss = F.cross_entropy(logits, y_true)
print("\nCross-Entropy (PyTorch):", ce_loss.item())

# === 5. Verificación manual (solo demostrativa, usando log-softmax) ===
log_probs = torch.log_softmax(logits, dim=1)
ce_manual = -log_probs[range(len(y_true)), y_true].mean()
print("Cross-Entropy (manual):", ce_manual.item())

# === 6. Comprobación de equivalencia numérica ===
assert torch.allclose(ce_loss, ce_manual, atol=1e-6)


Shapes → logits: (3, 3) | y_true: (3,)

Probabilidades (Softmax):
 tensor([[0.6590, 0.2424, 0.0986],
        [0.1086, 0.8025, 0.0889],
        [0.0493, 0.0545, 0.8962]])

Cross-Entropy (PyTorch): 0.24889366328716278
Cross-Entropy (manual): 0.24889366328716278
